# Get more fancy pants data
- go to the web factset 
- request it
- back to your sheet and run it (insert the name)


# Integrate it with the model
- join the data
- so update the model features


In [ ]:
https://app.snowflake.com/marketplace/listing/GZT0ZGCQ51UL/factset-tick-history

In [ ]:
snowpark group by stock/day,
max price min price mean price, rebuild ML model